# Compte-rendu - projet de génération d'un modèle de détection de mouvement de balancier

## TOC
* [Partie 1](#1)
* [Partie 2](#2)
* [Partie 3](#3)
* [Partie 4](#4)
* [Partie 5](#5)
* [Partie 6](#6)
* [Partie 7](#7)
* [Partie 8](#8)
* [Partie 9](#9)
* [Partie 10](#10)

## Initialisation <a class="anchor" id="1"></a>
Import des packages utile

* `pandas` : librairie pour manipuler les données (`DataFrame`)
* `numpy` : librairie mathématique
* `matplotlib` : librairie graphique

In [30]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [31]:
# Variables liées à l'environnement

# Fréquence d'échantillonnage de la carte
SENSORS_SAMPLING_RATE = 50 # Hz

## Récupération des données <a class="anchor" id="2"></a>

Nous avons au préalable réalisé des enregistrements de données des capteurs (disponibles dans le dossier `/data`).
Nous avons stocké dans un [Google sheet](https://docs.google.com/spreadsheets/d/1By59dQ56zL_kP0tW9Iyf4FppyEvJtcwnuG4gx1iokpM/edit?usp=sharing) si les captures correspondent à un état de balancier, ou non.

Nous devons traiter cette donnée brute pour la rendre plus compréhensible et interprétable par Keras.

### Trim

La première étape est de trim les enregistrements pour ne garder que la partie qui nous intéresse.
Cela est surtout important dans le cas où notre capture représente un mouvement de balancier, car la donnée brute inclut, au début et à la fin, des instants où l'objet ne se balance pas. Cela correspond au temps qu'il s'écoule entre l'activation de la capture et le début du mouvement.

In [32]:
# Limites des données utiles de chaque fichier .csv
LIMITES = [ [60, 650],   [50, 350],  [100, 600],
            [0, 550],    [0, 550],   [150, 250],
            [0, 700],    [75, 820],  [0, 500],
            [50, 550],   [50, 750],  [50, 900],
            [0, 900],    [50, 1000], [100, 1200],
            [50, 1300],  [50, 900],  [200, 700],
            [0, 60],     [0, 60],    [0, 25],
            [100, 800],  [100, 900], [100, 600],
            [100, 850],  [50, 900],  [100, 1500],
            [100, 2200], [30, 300],  [0, 650],
            [0, 550],    [100, 500], [0, 710]]

len(LIMITES)

33

In [33]:
dataset = []

def trimDataset(ds):
    for i in range(len(ds)):
        ds[i] = ds[i][LIMITES[i][0]:LIMITES[i][1]]


for i in range(len(LIMITES)):
    dataset.append(pd.read_csv("data/balancier" + str(i) + ".csv"))

trimDataset(dataset)

len(dataset)

33

Le dataset contient maintenant une liste de dataframe de donnes brutes réduites à juste la partie qui nous intéresse.

### Echantillonnage

Dans cette partie nous allons échantillonner les données, c'est à dire que nous allons réduire le nombre de point de données pour un intervalle de temps donné.

**Raisonnement**

En se basant sur le [théorème de Shannon](), on peut déduire que la fréquence d'échantillonnage nécessaire est deux fois plus grande que la fréquence du signal à détecter.
Dans notre cas on a donc simplement besoin d'un échantillonnage de deux fois la fréquence maximale de balancier.
De par nos expériences on considère que le balancier maximal possible est de 5 Hz.
Notre échantillonnage sera donc de **10 Hz**.

**Intérêt**

cette méthode a deux intérêts :
* Réduire le nombre de points de données, et donc réduire le nombre de neurones d'entrée de notre modèle. Cela permet de réduire le temps de calcul.
* Chaque enregistrement contient plus de données que nécessaire, on peut donc les sous-diviser en plusieurs échantillons. Cela permet d'augmenter la data à notre disposition. Ici on multiplie la taille de notre dataset par 5.

In [34]:
DATA_SAMPLING_RATE = 10 # Hz

La fonction suivante `sampleDf` permet de sous-diviser une dataframe en plusieurs sous-échantillons.

In [35]:
def sampleDf(df, scaleFactor):
    res = []
    for i in range(0, scaleFactor): # A tester
        res.append(df.iloc[lambda x: x.index % scaleFactor == i])
    return res

In [39]:
# test de la fonction sampleDf
d = dataset[2]
scalingFactor = SENSORS_SAMPLING_RATE // DATA_SAMPLING_RATE

res = sampleDf(d, scalingFactor)


,T [ms],AccX [mg],AccY [mg],AccZ [mg],GyroX [mdps],GyroY [mdps],GyroZ [mdps]
100,39535,-5,670,-198,-10080,45710,-178080
105,39635,158,1997,-1584,253610,-889280,-1090040
110,39735,-162,1997,-1335,-76650,-814100,-665140
115,39835,-15,1893,-584,108010,-528150,-433160
120,39935,-113,75,-25,-330680,-272160,91840


_________________________________________________________

# Analyse des données
On commence par importer les données depuis un fichier CSV

In [ ]:
d = pd.read_csv('data/old/SensorTile_Log_N008.csv')
d.columns

In [ ]:
def cleanDataframe(df):
    df = df.drop(columns=["T [ms]"])
    return df

def sliceDf(df, step):
    res = []
    while (len(df) > step):
        res.append(df.iloc[:step])
        df = df.iloc[step:]
    return res

In [ ]:
#d = cleanDataframe(d)
d.tail(10)

In [ ]:
dfs = sliceDf(d, 100)
len(d)

In [ ]:
len(dfs)

In [ ]:
plt.plot(d["AccX [mg]"])
plt.plot(d["AccY [mg]"])
plt.plot(d["AccZ [mg]"])

In [ ]:
plt.plot(d["GyroX [mdps]"])

# Fetch balancier data depuis Gsheet

Récupération du résultat attendu depuis un Google Sheet

In [ ]:
from gSheet import SheetAPI

In [ ]:
# The ID and range of a sample spreadsheet.
SPREADSHEET_ID = '1By59dQ56zL_kP0tW9Iyf4FppyEvJtcwnuG4gx1iokpM'
api = SheetAPI(SPREADSHEET_ID)
api.connect()
print(api.getValues("A2:D100"))

# Parsing the data

Given the data, we can parse it to extract the information we need.

First we slice the dataframe into multiple 1-sec **rolling** windows
Then we multiply the data by sampling the data into subsets.

In [ ]:
SAMPLE_RATE = 50 # Hz
DATA_RATE = 20 # Hz
WINDOW_LENGTH = 2000 # (in ms)

In [ ]:

def sliceDf(df, step):
    res = []
    while (len(df) > step):
        res.append(df.iloc[:step])
        df = df.iloc[1:]
    return res

def removeTime(df):
    df = df.drop(columns=["T [ms]"])
    return df

def sampleDf(df, sample):
    res = []
    for i in range(0, sample): # A tester
        res.append(df.iloc[lambda x: x.index % sample == i])
    return res

In [ ]:
d = pd.read_csv('data/balancier0.csv')
d.columns

In [ ]:
d.tail(10)